# YOLO + ResNet 파이프라인 설명 (.GIF (or MP4) 처리 설명은 X)

이 파이프라인은 실시간 비디오 파일(임시적)에서 0.1초 당 1개 이미지 캡처 후,  
YOLO 모델을 활용하여 개 객체를 탐지한 후, 해당 개의 이미지를 ResNet 모델로 입력하여  
행동을 예측하고 로그로 저장하는 방식으로 작동합니다.  

파일 및 폴더 구조
- `ROGUN/`
  - `screenshot/` : YOLO 적용 전 원본 스크린샷 저장
  - `bbox_screenshot/` : YOLO를 통해 바운딩 박스를 표시한 이미지 저장
  - `log.csv` : 타임스탬프, 바운딩 박스 좌표, 예측된 행동 클래스 저장

> 이 폴더와 파일들은 자동으로 생성됩니다

---

파이프라인 동작 원리

1. 실시간 프레임 캡처
- OpenCV를 사용하여 0.1초마다 프레임을 캡처.
- `screenshot/` 폴더에 원본 프레임을 저장.

2. YOLO를 이용한 개 감지
- YOLO 모델을 사용하여 개 객체만 탐지.
- 개가 여러 마리 감지될 경우, 가장 높은 신뢰도를 가진 개 한 마리를 선택.

3. 바운딩 박스를 그린 이미지 저장
- 개가 감지된 경우, 원본 이미지 위에 바운딩 박스를 그린 후 `bbox_screenshot/` 폴더에 저장.
- 나중에 .gif 만드는데 사용.

4. ResNet을 활용한 행동 분류
- YOLO로 감지된 개의 이미지를 크롭하여 ResNet 모델에 입력.
- 크롭된 이미지는 따로 저장 안되고 메모리에서 바로 넘어옴.
- ResNet 모델은 10가지 행동 클래스 중 예측.

5. 예측 결과를 CSV 로그에 저장
- `log.csv` 파일에 타임스탬프, 바운딩 박스 좌표, 예측된 행동 클래스를 저장한다.

---

파이프라인 실행 예시

개가 감지된 경우
- 스크린샷 저장: `ROGUN/screenshot/20250222-143050.jpg`
- 개 감지됨 (좌표: (120, 80, 250, 220), 신뢰도: 0.85)
- 바운딩 박스 이미지 저장: `ROGUN/bbox_screenshot/20250222-143050_bbox.jpg`
- 예측된 포즈: WALKRUN (확률: 0.92)
- 로그 저장 완료: `ROGUN/log.csv`

활용 예시

- 비디오 파일 분석
  - `video_path = "/path/to/video.mp4"` (비디오 파일 경로 설정)
- YOLO v11 사용 
  - `yolo_model = YOLO("yolov11m.pt")` 
  - 속도가 느리면 더 빠른 욜로 모델로 변경 가능 
- **ResNet18 → ResNet50 변경 가능**
  - `resnet_model = models.resnet50(weights=None)`

---

시스템 요구사항
- Python 3.8 이상
- PyTorch, OpenCV, PIL, Pandas, Ultralytics (YOLO)
- Apple MPS (Mac) 또는 CUDA (Windows/Linux)


In [ ]:
#영상에서 스크린샷 저장, bbox 스크린샷 저장, csv 로그 저장 하는 코드고 실시간 영상 처리하는 코드는 아닙니다.
#.MP4 파일 만들어주는 코드 다음 코드에 있습니다. .GIF 보다 .MP4 파일이 더 크기가 작아서 일단 .MP4로 만들었습니다(쉽게 .GIF로 변경가능).
#GIF 또는 MP4 파일이 생각보다 고려할게 많아서 다음 코드 참고하시고 기능 유지할지 고쳐 사용할지는 여러분 마음입니다. 저는 둘다 찬성입니다.

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
import cv2
import numpy as np
import pandas as pd
import time
import os
from ultralytics import YOLO
from PIL import Image

# ------------------------
# 1. 환경 설정
# ------------------------
video_path = "/Users/vairocana/Downloads/test_video1.mp4"  # 실시간 스트리밍을 사용할 경우, 0으로 설정 (video_path = 0). 경로는 필요시 변경
output_dir = "/Users/vairocana/Downloads/ROGUN"  # 저장할 폴더
screenshot_dir = os.path.join(output_dir, "screenshot")  # 원본 스크린샷 저장 폴더
bbox_screenshot_dir = os.path.join(output_dir, "bbox_screenshot")  # 바운딩 박스 스크린샷 저장 폴더
log_csv_path = os.path.join(output_dir, "log.csv")  # CSV 로그 경로

# 폴더 생성 (존재하지 않으면 자동 생성)
os.makedirs(screenshot_dir, exist_ok=True)
os.makedirs(bbox_screenshot_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# CSV 로그 초기화 (없으면 새로 생성)
columns = ["timestamp", "bbox", "class"]
log_df = pd.DataFrame(columns=columns)

# ------------------------
# 2. YOLOv11 모델 로드 (개 객체 탐지) ***욜로 모델은 직접 다운 받고 경로 설정해야함***
# ------------------------
yolo_model = YOLO("/Users/vairocana/Downloads/yolo11m.pt")  #YOLOv11m 사용. 느리면 11s, 11n 사용.
print("✅ YOLO 모델이 성공적으로 로드되었습니다!")

# ------------------------
# 3. ResNet 모델 로드 (포즈 분류) ***학습시킨 모델이 resnet50이라면 resnet50으로 변경***
# ------------------------
resnet_model = models.resnet18(weights=None) #필요시 변경
num_features = resnet_model.fc.in_features
num_classes = 10  # 현재 10개 클래스 사용. 8개 클래스 모델이면 바꿔야 함.
resnet_model.fc = nn.Linear(num_features, num_classes)

# 학습된 모델 가중치 불러오기
model_path = "/Users/vairocana/Desktop/AI/resnet_models/resnet18_model_82.pth" #경로는 각자 설정
resnet_model.load_state_dict(torch.load(model_path, map_location="cpu"))
resnet_model.to("cpu")
resnet_model.eval()  # 평가 모드 (Dropout, BatchNorm 비활성화)
print("✅ ResNet 모델이 성공적으로 로드되었습니다!")

# ------------------------
# 4. 이미지 전처리 함수 (ResNet 입력에 맞게 변환)
# ------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 224x224 크기로 조정
    transforms.ToTensor(),  # PyTorch 텐서 변환
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet 정규화
])

# ------------------------
# 5. 실시간 영상 처리 (YOLO → ResNet + 이미지 저장)
# ------------------------
def process_video():
    global log_df  # CSV 로그 데이터프레임

    cap = cv2.VideoCapture(video_path)  # OpenCV를 사용하여 비디오 스트림 열기
    if not cap.isOpened():
        print("❌ 비디오를 열 수 없습니다!")
        return
    
    frame_count = 0  # 프레임 카운터 초기화

    while cap.isOpened():
        ret, frame = cap.read()  # 프레임 읽기
        if not ret:
            print("⏹️ 영상 스트림이 끝났거나, 오류 발생")
            break
        
        frame_count += 1
        
        # 0.1초(100ms)마다 처리 (30 FPS 기준, 약 3프레임마다 1번)
        if frame_count % 3 != 0:
            continue

        timestamp = time.strftime("%Y%m%d-%H%M%S")  # 현재 타임스탬프 기록
        
        # OpenCV는 BGR, YOLO는 RGB 사용하므로 변환
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 원본 스크린샷 저장
        screenshot_path = os.path.join(screenshot_dir, f"{timestamp}.jpg")
        cv2.imwrite(screenshot_path, frame)  
        print(f"📸 스크린샷 저장: {screenshot_path}")

        # ------------------------
        # 6. YOLO 모델 실행 (개 탐지)
        # ------------------------
        results = yolo_model(frame_rgb)
        best_box = None  # 가장 신뢰도 높은 바운딩 박스
        best_confidence = 0.0  # 최고 신뢰도 초기화

        for result in results:
            for box in result.boxes.data:
                x1, y1, x2, y2, conf, cls = box.tolist()
                
                if int(cls) == 16 and conf > best_confidence:  # 클래스 16은 'dog'
                    best_confidence = conf
                    best_box = (int(x1), int(y1), int(x2), int(y2))

        # 개가 감지되지 않으면 해당 프레임 건너뜀
        if best_box is None:
            #print(f"⏩ 개가 감지되지 않음 (Timestamp: {timestamp})")
            continue

        x1, y1, x2, y2 = best_box
        print(f"🐶 개 감지됨 (좌표: {best_box}, 신뢰도: {best_confidence:.2f})")

        # 바운딩 박스 그린 이미지 저장
        bbox_screenshot_path = os.path.join(bbox_screenshot_dir, f"{timestamp}_bbox.jpg")
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # 빨간색 사각형 그리기
        cv2.imwrite(bbox_screenshot_path, frame)
        print(f"📸 바운딩 박스 이미지 저장: {bbox_screenshot_path}")

        # ------------------------
        # 7. 바운딩 박스 이미지 ResNet에 입력
        # ------------------------
        cropped_img = frame_rgb[y1:y2, x1:x2]  # YOLO에서 탐지된 영역 크롭
        cropped_img_pil = Image.fromarray(cropped_img)  # PIL 이미지 변환
        processed_img = transform(cropped_img_pil)  # 전처리 적용
        processed_img = processed_img.unsqueeze(0)  # 배치 차원 추가

        # ResNet 모델로 분류 수행
        with torch.no_grad():
            outputs = resnet_model(processed_img)
            probs = torch.softmax(outputs, dim=1)  # 확률값 계산
            predicted_class = torch.argmax(probs, dim=1).item()  # 가장 높은 확률의 클래스 선택
            confidence = probs[0, predicted_class].item()  # 해당 클래스의 확률값

        # ------------------------
        # 8. CSV 로그 저장
        # ------------------------
        class_labels = [
            "BODYLOWER", "BODYSCRATCH", "BODYSHAKE", "FEETUP",
            "FOOTUP", "LYING", "MOUNTING", "SIT", "TURN", "WALKRUN"
        ]
        predicted_label = class_labels[predicted_class]
        print(f"✅ 예측된 포즈: {predicted_label} (확률: {confidence:.2f})")

        # 8. CSV 로그 저장 (즉시 파일에 추가)
        new_log = pd.DataFrame([[timestamp, best_box, predicted_label]], columns=log_df.columns)
        new_log.to_csv(log_csv_path, mode='a', header=not os.path.exists(log_csv_path), index=False)  # ✅ 즉시 CSV 저장
        print(f"📂 즉시 CSV 저장 완료: {log_csv_path}")


    cap.release()  # 비디오 스트림 해제
    log_df.to_csv(log_csv_path, index=False)
    print("📂 CSV 로그 저장 완료:", log_csv_path)

# ------------------------
# 9. 실행
# ------------------------
if __name__ == "__main__":
    process_video()


### 아래 코드는 영상에서 원본 스크린샷, BBOX 된 스크린샷, log.csv(timestamp, bbox, class), 클래스 변경시.MP4까지 만드는 전체 코드입니다. 고칠거 고치고 사용하면 될거 같습니다.

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
import cv2
import numpy as np
import pandas as pd
import time
import os
import glob
from ultralytics import YOLO
from PIL import Image

# ------------------------
# 1. 환경 설정
# ------------------------
video_path = "/Users/vairocana/Downloads/test_video1.mp4"  # 실시간 스트리밍 사용 시 video_path = 0
output_dir = "/Users/vairocana/Downloads/ROGUN"  # 저장할 폴더
screenshot_dir = os.path.join(output_dir, "screenshot")  # 원본 스크린샷 저장 폴더
bbox_screenshot_dir = os.path.join(output_dir, "bbox_screenshot")  # 바운딩 박스 스크린샷 저장 폴더
mp4_output_dir = os.path.join(output_dir, "mp4")  # MP4 저장 폴더
log_csv_path = os.path.join(output_dir, "log.csv")  # CSV 로그 경로

# 폴더 생성 (존재하지 않으면 자동 생성)
os.makedirs(screenshot_dir, exist_ok=True)
os.makedirs(bbox_screenshot_dir, exist_ok=True)
os.makedirs(mp4_output_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# CSV 로그 초기화 (없으면 새로 생성)
columns = ["timestamp", "bbox", "class"]
log_df = pd.DataFrame(columns=columns)

# ------------------------
# 2. YOLOv11 모델 로드 (개 객체 탐지)
# ------------------------
yolo_model = YOLO("/Users/vairocana/Downloads/yolo11m.pt")  # YOLOv11m 사용
print("✅ YOLO 모델 로드 완료!")

# ------------------------
# 3. ResNet 모델 로드 (포즈 분류)
# ------------------------
resnet_model = models.resnet18(weights=None)  # 필요시 resnet50으로 변경 가능
num_features = resnet_model.fc.in_features
num_classes = 10  # 현재 10개 클래스 사용. 8개 클래스 모델이면 바꿔야 함.
resnet_model.fc = nn.Linear(num_features, num_classes)

# 학습된 모델 가중치 불러오기
model_path = "/Users/vairocana/Desktop/AI/resnet_models/resnet18_model_82.pth"  # 경로 수정 필요
resnet_model.load_state_dict(torch.load(model_path, map_location="cpu"))
resnet_model.to("cpu")
resnet_model.eval()  # 평가 모드 설정
print("✅ ResNet 모델 로드 완료!")

# ------------------------
# 4. 이미지 전처리 함수 (ResNet 입력에 맞게 변환)
# ------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 224x224 크기로 조정
    transforms.ToTensor(),  # PyTorch 텐서 변환
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet 정규화
])

# ------------------------
# 5. MP4 저장 함수 (클래스 변경 감지 시 실행)
# ------------------------
def save_behavior_change_mp4(timestamp, prev_class, new_class):
    """
    행동 변화 감지 시 MP4 비디오를 생성하여 저장하는 함수
    - timestamp: 클래스 변화가 감지된 시점
    - prev_class: 변경 전 행동 클래스
    - new_class: 변경 후 행동 클래스
    """
    mp4_filename = f"{prev_class}_TO_{new_class}_{timestamp}.mp4"
    mp4_path = os.path.join(mp4_output_dir, mp4_filename)

    # 스크린샷 폴더에서 모든 이미지 파일 목록 불러오기
    all_images = sorted(glob.glob(os.path.join(screenshot_dir, "*.jpg")))

    # 현재 timestamp와 가장 가까운 인덱스 찾기
    target_index = None
    for i, img_path in enumerate(all_images):
        if timestamp in img_path:
            target_index = i
            break

    if target_index is None:
        print(f"⚠️ {timestamp} 기준으로 프레임을 찾을 수 없음!")
        return

    # 이전 50개 + 이후 50개 → 총 100개 프레임 선택
    start_idx = max(0, target_index - 50)
    end_idx = min(len(all_images), target_index + 50)
    selected_images = all_images[start_idx:end_idx]

    if len(selected_images) < 10:  # 최소 프레임 수 제한
        print(f"⚠️ MP4 생성 실패: 프레임 수 부족 ({len(selected_images)}개)")
        return

    # 첫 번째 이미지로 해상도 확인
    first_frame = cv2.imread(selected_images[0])
    height, width, _ = first_frame.shape

    # OpenCV VideoWriter 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 코덱 설정
    fps = 10  # 초당 10프레임 설정
    video_writer = cv2.VideoWriter(mp4_path, fourcc, fps, (width, height))

    # 선택된 이미지들을 하나씩 비디오 프레임으로 추가
    for img_path in selected_images:
        frame = cv2.imread(img_path)
        if frame is not None:
            video_writer.write(frame)

    video_writer.release()
    print(f"🎥 MP4 저장 완료: {mp4_path}")

# ------------------------
# 6. 실시간 영상 처리
# ------------------------
def process_video():
    global log_df  # CSV 로그 데이터프레임

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("❌ 비디오를 열 수 없습니다!")
        return
    
    frame_count = 0  

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("⏹️ 영상 스트림 종료")
            break
        
        frame_count += 1
        
        if frame_count % 3 != 0:
            continue

        timestamp = time.strftime("%Y%m%d-%H%M%S")
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        screenshot_path = os.path.join(screenshot_dir, f"{timestamp}.jpg")
        cv2.imwrite(screenshot_path, frame)  

        results = yolo_model(frame_rgb)
        best_box, best_confidence = None, 0.0

        for result in results:
            for box in result.boxes.data:
                x1, y1, x2, y2, conf, cls = box.tolist()
                
                if int(cls) == 16 and conf > best_confidence:
                    best_confidence = conf
                    best_box = (int(x1), int(y1), int(x2), int(y2))

        if best_box is None:
            continue

        x1, y1, x2, y2 = best_box

        bbox_screenshot_path = os.path.join(bbox_screenshot_dir, f"{timestamp}_bbox.jpg")
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.imwrite(bbox_screenshot_path, frame)

        cropped_img = frame_rgb[y1:y2, x1:x2]
        cropped_img_pil = Image.fromarray(cropped_img)
        processed_img = transform(cropped_img_pil).unsqueeze(0)

        with torch.no_grad():
            outputs = resnet_model(processed_img)
            predicted_class = torch.argmax(torch.softmax(outputs, dim=1), dim=1).item()

        class_labels = ["BODYLOWER", "BODYSCRATCH", "BODYSHAKE", "FEETUP", "FOOTUP", "LYING", "MOUNTING", "SIT", "TURN", "WALKRUN"]
        predicted_label = class_labels[predicted_class]

        if os.path.exists(log_csv_path):
            existing_log = pd.read_csv(log_csv_path)
            if len(existing_log) > 0 and existing_log.iloc[-1]["class"] != predicted_label:
                save_behavior_change_mp4(timestamp, existing_log.iloc[-1]["class"], predicted_label)

        pd.DataFrame([[timestamp, best_box, predicted_label]], columns=log_df.columns).to_csv(log_csv_path, mode='a', header=not os.path.exists(log_csv_path), index=False)

if __name__ == "__main__":
    process_video()


# ---------------------------------------------------------------------

## 여기서 부터 전에 사용하던 기본 코드


# YOLO v11s를 이용한 개 객체 탐지 후 224x224로 리사이즈하고 모델 테스트
욜로는 COCO 데이터셋의 80개의 클래스를 가지고 사전학습된 모델입니다.
80개의 클래스 중에 16번 인덱스(숫자로 세면 17번쨰)이 개 클래스인데 출처에 따라 인덱스 17이라고 조금 다르게 나옵니다.
일단 작동해서 16으로 두었습니다.

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import cv2
import numpy as np
from ultralytics import YOLO

# 모델, 욜로, 이미지 경로 수정만 해주시면 됩니다.
# ------------------------
# 1. YOLOv11 모델 로드 (개 객체 탐지용)
# ------------------------
# YOLOv11s 모델은 COCO 데이터셋으로 사전 학습되어 있으므로, 'dog' 클래스(일반적으로 클래스 인덱스 16)를 인식합니다.
yolo_model_path = "/Users/vairocana/Downloads/yolo11m.pt"  # YOLOv11m 모델 파일 경로 (해당 파일이 존재해야 함)
yolo_model = YOLO(yolo_model_path)
print("✅ YOLOv11 모델이 성공적으로 로드되었습니다!")

# ------------------------
# 2. ResNet 모델 로드 (분류용)
# ------------------------
model_path = "/Users/vairocana/Desktop/AI/resnet_models/resnet18_model_82.pth"  # 저장된 ResNet50 모델 파일 경로
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print("Using device:", device)

# ResNet50 모델을 사전 학습된 가중치 없이 생성한 후, 마지막 fc 레이어를 데이터셋 클래스 수에 맞게 수정
model = models.resnet18(weights=None)
num_features = model.fc.in_features
num_classes = 10  # 클래스 개수 (데이터셋에 맞게 수정)
model.fc = nn.Linear(num_features, num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()  # 평가 모드로 전환 (Dropout, BatchNorm 등 비활성화)
print("✅ ResNet모델이 성공적으로 로드되었습니다!")

# ------------------------
# 3. 이미지 전처리 함수 정의 (224×224, 정규화)
# ------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 크롭된 이미지를 224×224로 조정
    transforms.ToTensor(),  # 텐서 변환
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet 정규화
])

# ------------------------
# 4. YOLO를 이용하여 "dog" 객체를 탐지하고 크롭하는 함수
# ------------------------
def detect_and_crop(image_path):
    """
    주어진 이미지 경로에서 YOLOv8을 이용해 'dog' 객체(클래스 인덱스 16)를 탐지하고,
    해당 바운딩 박스 영역을 크롭한 후 224×224로 전처리한 이미지를 반환합니다.
    """
    # 이미지 로드 및 BGR -> RGB 변환 (YOLO는 RGB 사용)
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ 이미지 로드 실패: {image_path}")
        return None
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # YOLO 추론 수행
    results = yolo_model(image_rgb)

    # 디버그: 탐지된 모든 객체 정보 출력인데 개 객체만 출력하려면
    print("🔍 검출된 객체들:")
    for result in results:
        for box in result.boxes.data:
            # YOLOv11 결과 tensor를 리스트로 변환하여 [x1, y1, x2, y2, confidence, class] 순서로 얻음
            box_vals = box.tolist()
            x1, y1, x2, y2, conf, cls = box_vals
            print(f"  클래스: {int(cls)}, 신뢰도: {conf:.2f}")

    # 'dog' 객체 추출 (COCO 기준 dog 클래스 인덱스는 일반적으로 16)
    found_dog = False
    for result in results:
        for box in result.boxes.data:
            box_vals = box.tolist()
            x1, y1, x2, y2, conf, cls = box_vals
            # 여기서 dog 클래스 조건: 클래스 인덱스 16, 신뢰도 0.4 이상 (필요에 따라 조정)
            if int(cls) == 16 and conf > 0.4:
                found_dog = True
                # 좌표 정수형 변환
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                # 개 객체 영역 크롭 (RGB 이미지 기준)
                cropped_img = image_rgb[y1:y2, x1:x2]
                # PIL 이미지로 변환
                cropped_img_pil = Image.fromarray(cropped_img)
                # 전처리: 224×224 리사이즈 및 정규화, 배치 차원 추가
                processed_img = transform(cropped_img_pil)
                processed_img = processed_img.unsqueeze(0)
                return processed_img.to(device)
    if not found_dog:
        print("⚠️ 이미지에서 개 객체를 감지하지 못했습니다!")
    return None

# ------------------------
# 5. ResNet을 이용하여 포즈(클래스) 분류하는 함수
# ------------------------
def classify_pose(image_path):
    """
    YOLO를 사용해 주어진 이미지에서 개 객체를 탐지하여 크롭한 후,
    ResNet 모델로 분류하여 포즈(클래스)를 예측하는 함수.
    """
    cropped_image = detect_and_crop(image_path)
    if cropped_image is None:
        print("❌ 예측을 수행할 개 이미지가 없습니다.")
        return

    # ResNet50을 이용한 예측 (gradient 계산 비활성화)
    with torch.no_grad():
        outputs = model(cropped_image)
        _, predicted_class = outputs.max(1)

    # 클래스 라벨 (데이터셋에 맞게 수정)
    class_labels = [
        "BODYLOWER", "BODYSCRATCH", "BODYSHAKE", "FEETUP", 
        "FOOTUP", "LYING", "MOUNTING", "SIT", "TURN", "WALKRUN"
    ]
    predicted_label = class_labels[predicted_class.item()]
    print(f"✅ 예측된 포즈: {predicted_label}")

# ------------------------
# 6. 실행: 단일 이미지에 대해 YOLO로 개 객체 탐지 후, ResNet으로 분류
# ------------------------
image_path = "/Users/vairocana/Downloads/sample7.jpeg"  # 테스트할 이미지 경로 (수정 필요)
classify_pose(image_path)


✅ YOLOv11 모델이 성공적으로 로드되었습니다!
Using device: mps


RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([10, 512]) from checkpoint, the shape in current model is torch.Size([8, 512]).
	size mismatch for fc.bias: copying a param with shape torch.Size([10]) from checkpoint, the shape in current model is torch.Size([8]).

# 욜로로 크롭된 이미지 확인을 위한 코드

In [44]:
import torch
import cv2
import os
from ultralytics import YOLO
from PIL import Image
import numpy as np

#욜로와 이미지 경로만 수정해주시면 됩니다.
# ------------------------
# 1. YOLOv11 모델 로드 
# ------------------------
yolo_model_path = "/Users/vairocana/Downloads/yolo11s.pt"  # YOLOv11s 모델 파일 경로 (해당 파일이 존재해야 함)
yolo_model = YOLO(yolo_model_path)
print("✅ YOLOv11 모델이 성공적으로 로드되었습니다!")

# ------------------------
# 2. 이미지 경로 설정 (필요에 따라 수정)
# ------------------------
image_path = "/Users/vairocana/Downloads/coco_sample2.jpg"
output_path = "/Users/vairocana/Downloads//cropped_dog.jpg"

# ------------------------
# 3. 이미지 로드 및 YOLO 추론 수행
# ------------------------
image = cv2.imread(image_path)
if image is None:
    print(f"❌ 이미지 로드 실패: {image_path}")
    exit()

# OpenCV는 BGR, YOLO는 RGB이므로 변환
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

results = yolo_model(image_rgb)  # YOLO 추론 수행

# ------------------------
# 4. 디버깅: 검출된 모든 객체 정보 출력
# ------------------------
print("🔍 검출된 객체들:")
for result in results:
    # result.boxes.data는 각 박스에 대해 [x1, y1, x2, y2, confidence, class]를 포함하는 tensor입니다.
    for box in result.boxes.data:
        # 텐서를 리스트로 변환하여 인덱스로 접근
        box = box.tolist()
        x1, y1, x2, y2, conf, cls = box
        print(f"  클래스: {int(cls)}, 신뢰도: {conf:.2f}")

# ------------------------
# 5. "dog" 클래스 (COCO 기준 dog의 클래스 인덱스는 16이라고 나온곳도 있고 17도 있음..)만 추출하여 크롭
# ------------------------
found_dog = False
for result in results:
    for box in result.boxes.data:
        box = box.tolist()
        x1, y1, x2, y2, conf, cls = box
        if int(cls) == 16 and conf > 0.4:  # dog 클래스 인덱스가 16 아니면 17인데 찾아도 둘다 나와서 일단 16으로 설정. 조건 (신뢰도 0.4 이상)
            found_dog = True
            # 좌표 정수형 변환
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            # 개 객체 영역 크롭 (RGB 이미지 기준)
            cropped_dog = image_rgb[y1:y2, x1:x2]
            # PIL 이미지로 변환 후 저장 (JPEG 품질 95)
            cropped_img_pil = Image.fromarray(cropped_dog)
            cropped_img_pil.save(output_path, quality=95)
            print(f"✅ 개 객체를 검출하여 크롭했습니다! 저장 경로: {output_path}")
            break  # 하나의 개만 처리
    if found_dog:
        break

if not found_dog:
    print("❌ 이미지에서 개 객체를 감지하지 못했습니다!")


✅ YOLOv11 모델이 성공적으로 로드되었습니다!

0: 640x512 2 dogs, 1 skis, 46.3ms
Speed: 1.3ms preprocess, 46.3ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 512)
🔍 검출된 객체들:
  클래스: 16, 신뢰도: 0.81
  클래스: 16, 신뢰도: 0.36
  클래스: 30, 신뢰도: 0.33
✅ 개 객체를 검출하여 크롭했습니다! 저장 경로: /Users/vairocana/Downloads//cropped_dog.jpg
